In [ ]:
import pandas as pd
import unicodedata

# --- utilidades ---
def quitar_acentos(s):
    if pd.isna(s): 
        return s
    return ''.join(c for c in unicodedata.normalize('NFKD', str(s)) if not unicodedata.combining(c))

def to_Int64(series):
    s = pd.to_numeric(series, errors="coerce")
    return s.round().astype("Int64")

def to_float64(series):
    return pd.to_numeric(series, errors="coerce").astype("float64")

# =========================
# 1) CARGA
# =========================
# r"C:\Users\Usuario\Desktop\Maestria\Programacion\BigDataUBA-Grupo1\TP1\Bases de datos\usu_individual_T125.xlsx"
# r"C:\Users\Usuario\Desktop\Maestria\Programacion\BigDataUBA-Grupo1\TP1\Bases de datos\Individual_t105.dta"

ruta_dta  = r"C:\Users\AGUSTIN\Desktop\Individual_t105.dta"      # 2005
ruta_xlsx = r"C:\Users\AGUSTIN\Desktop\usu_individual_T125.xlsx" # 2025

base_2005 = pd.read_stata(ruta_dta)
base_2025 = pd.read_excel(ruta_xlsx)

# =========================
# 2) FILTRO POR REGIÓN
# =========================
# 2005: región textual -> quedarnos solo "Patagonica/Patagónica"
if "region" in base_2005.columns:
    reg_txt = base_2005["region"].astype("string").map(quitar_acentos).str.lower()
    base_2005 = base_2005[reg_txt == "patagonica"]

# 2025: región numérica == 44
if "REGION" in base_2025.columns:
    base_2025 = base_2025[base_2025["REGION"] == 44]

# =========================
# 3) VARIABLES A CONSERVAR
# =========================
vars_identif = ["CODUSU","NRO_HOGAR","COMPONENTE","H15","ANO4","TRIMESTRE","REGION","PONDERA"]
vars_analit  = ["CH04","CH06","CH07","CH08","NIVEL_ED","ESTADO","CAT_INAC","IPCF",
                "CH15","PP02H","PP10A","PP10E","PP11A","PP11O","TOT_P12"]
vars_keep    = vars_identif + vars_analit

# Normalizar nombres a MAYÚSCULAS
base_2005.columns = [c.upper() for c in base_2005.columns]
base_2025.columns = [c.upper() for c in base_2025.columns]

# Filtrar solo columnas disponibles
base_2005 = base_2005[[c for c in vars_keep if c in base_2005.columns]]
base_2025 = base_2025[[c for c in vars_keep if c in base_2025.columns]]

# =========================
# 4) ARMONIZAR TIPOS A ESQUEMA 2025
# =========================
# Tipos objetivo (según tu XLSX 2025)
int_cols_target   = ["CH04","CH06","CH07","NIVEL_ED","ESTADO","CAT_INAC","CH15","PP02H","TOT_P12",
                     "NRO_HOGAR","COMPONENTE","H15","ANO4","TRIMESTRE","REGION","PONDERA"]
float_cols_target = ["CH08","IPCF","PP10A","PP10E","PP11A","PP11O"]
string_cols_target= ["CODUSU"]

# En 2005, REGION es textual. Ya filtramos Patagonia; ahora seteamos REGION=44 para matchear 2025
if "REGION" in base_2005.columns:
    base_2005["REGION"] = 44

# Crear columnas faltantes en 2005 si hiciera falta (con NA)
for col in vars_keep:
    if col not in base_2005.columns:
        base_2005[col] = pd.NA

# Conversiones en 2005
for col in base_2005.columns:
    if col in string_cols_target:
        base_2005[col] = base_2005[col].astype("string")
    elif col in int_cols_target:
        base_2005[col] = to_Int64(base_2005[col])
    elif col in float_cols_target:
        base_2005[col] = to_float64(base_2005[col])

# Para 2025: asegurar tipos también (por si vinieron raros desde Excel)
for col in base_2025.columns:
    if col in string_cols_target:
        base_2025[col] = base_2025[col].astype("string")
    elif col in int_cols_target:
        base_2025[col] = to_Int64(base_2025[col])
    elif col in float_cols_target:
        base_2025[col] = to_float64(base_2025[col])

# Reordenar columnas en el mismo orden
base_2005 = base_2005[vars_keep]
base_2025 = base_2025[vars_keep]

# =========================
# 5) CONTROL
# =========================
print("📊 DTA 2005 dtypes:\n", base_2005.dtypes)
print("\n📊 XLSX 2025 dtypes:\n", base_2025.dtypes)

comparacion = []
for col in vars_keep:
    if col in base_2005.columns and col in base_2025.columns:
        comparacion.append({
            "Variable": col,
            "DTA_2005": str(base_2005[col].dtype),
            "XLSX_2025": str(base_2025[col].dtype),
            "Coincide": str(base_2005[col].dtype) == str(base_2025[col].dtype)
        })
print("\n🔎 Comparación de dtypes (2005 vs 2025):")
print(pd.DataFrame(comparacion))

📊 DTA 2005 dtypes:
 CODUSU        string[python]
NRO_HOGAR              Int64
COMPONENTE             Int64
H15                    Int64
ANO4                   Int64
TRIMESTRE              Int64
REGION                 Int64
PONDERA                Int64
CH04                   Int64
CH06                   Int64
CH07                   Int64
CH08                 float64
NIVEL_ED               Int64
ESTADO                 Int64
CAT_INAC               Int64
IPCF                 float64
CH15                   Int64
PP02H                  Int64
PP10A                float64
PP10E                float64
PP11A                float64
PP11O                float64
TOT_P12                Int64
dtype: object

📊 XLSX 2025 dtypes:
 CODUSU        string[python]
NRO_HOGAR              Int64
COMPONENTE             Int64
H15                    Int64
ANO4                   Int64
TRIMESTRE              Int64
REGION                 Int64
PONDERA                Int64
CH04                   Int64
CH06            

In [3]:
# =========================
# 6) (Opcional) GUARDAR
# =========================
base_2005.to_excel(r"C:\Users\AGUSTIN\Desktop\EPH_individual_2005_armonizada.xlsx", index=False)
base_2025.to_excel(r"C:\Users\AGUSTIN\Desktop\EPH_individual_2025_filtrada.xlsx", index=False)


In [7]:
# Parte 3 | Fran

import pandas as pd
import unicodedata

base_i05 = pd.read_stata(r"C:\Trabajos\Bases de datos\Individual_t105.dta")
base_i25 = pd.read_excel(r"C:\Trabajos\Bases de datos\usu_individual_T125.xlsx")

base_h05 = pd.read_stata(r"C:\Trabajos\Bases de datos\Hogar_t105.dta")
base_h05 = pd.read_excel(r"C:\Trabajos\Bases de datos\usu_hogar_T125.xlsx")


#